In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold


from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from contrastive import CPCA	# $ pip3 install contrastive
import umap.umap_ as umap

# set for your own directory
my_filepath = "/Users/kinichen/Summer_dFC/Datasets/ds003465_task-Axcpt_Time-Freq.npy"

In [3]:
# Loading dataset for 1 task paradigm assessed by 1 method for all subjects (1 run)
dFC = np.load(my_filepath, allow_pickle=True)
dFC_dict = dFC.item() # extract the dictionary from np array

X = dFC_dict["X"]
y = dFC_dict["y"]
subj_label = dFC_dict["subj_label"]
method = dFC_dict["measure_name"]

In [4]:
def dim_reduction_train(X, model_class, n_components, Y=None, alpha=1):
	"""
	Fit dimension reduction pipeline on train data, and return both transformed X and the reusable pipeline.
	
	Parameters:
		X: array-like, shape (n_samples_train, n_features)
		model_class: class (PCA, CCA, CPCA, or UMAP)
		n_components: number of components to keep on outermost layer (not PCA if
			it is used as a preprocessing step)
		Y: for supervised CCA or CPCA background
		alpha: CPCA contrastive parameter
		
	Returns:
		X_reduced: transformed training data
		pipeline: object to reuse to transform test data
	"""
	
	scaler_X = StandardScaler()
	X_scaled = scaler_X.fit_transform(X)

	if model_class.__name__ == 'PCA':
		model = model_class(n_components=n_components)
		pipeline = Pipeline([
			('scaler', scaler_X),
			('pca', model)
		])
		X_reduced = pipeline.fit_transform(X)

	elif model_class.__name__ == 'CCA':
		# Note: it only makes sense to use supervised CCA with labels Y=y. If
		# using Y=X_rest, the reduced subspace does not enhance task-specific
		# features, but instead returns the shared subspace between task and rest.
		if Y is None:
			raise ValueError("CCA requires supervised labels Y.")

		# Preprocess labels into "2D" array with shape (n_samples, 1), then OneHotEncoder
		y = OneHotEncoder(sparse_output=False).fit_transform(Y.reshape(-1, 1))

		# PCA before CCA
		pca = PCA(n_components=1000)
		X_pca = pca.fit_transform(X_scaled)

		cca = model_class(n_components=n_components)
		X_cca, _ = cca.fit_transform(X_pca, y)

		# Store fitted pipeline parts manually into a dictionary
		pipeline = {
			'scaler': scaler_X,
			'pca': pca,
			'cca': cca,
			'label_encoder': y  # just for reference if needed
		}
		X_reduced = X_cca

	elif model_class.__name__ == 'CPCA':
		if Y is None:
			raise ValueError("CPCA requires background dataset Y.")
		
		scaler_Y = StandardScaler()
		Y_scaled = scaler_Y.fit_transform(Y)

		# Note: the CPCA class intrinsically applies PCA to reduce to 1000 
  		# components first, so for consistency on test data, do this explicitly
		pca = PCA(n_components=1000)
		X_pca = pca.fit_transform(X_scaled)
		Y_pca = pca.transform(Y_scaled)	# transform Y into the same PCA feature space

		cpca = model_class(n_components=n_components)
		X_cpca = cpca.fit_transform(X_pca, Y_pca, 
							alpha_selection='manual', alpha_value=alpha)
		
		pipeline = {
			'scaler_X': scaler_X,	# Don't store scaler_Y, as it is not needed for test data
			'pca': pca,
   			'cpca': cpca,
			'alpha': alpha 
		}
		X_reduced = X_cpca

	elif model_class.__name__ == 'UMAP':
		pca = PCA(n_components=1000)
		X_pca = pca.fit_transform(X_scaled)

		umap_model = umap.UMAP(n_neighbors=50, min_dist=0.1, 
							   n_components=n_components, random_state=25)
		X_umap = umap_model.fit_transform(X_pca)

		pipeline = {
			'scaler': scaler_X,
			'pca': pca,
			'umap': umap_model
		}
		X_reduced = X_umap

	else:
		raise ValueError("Only PCA, CCA, CPCA, UMAP are supported right now.")

	return X_reduced, pipeline


In [5]:
def dim_reduction_test(X, pipeline):
	"""
	Return the transformed test data using the fitted model pipeline.
	
	Parameters:
		X: array-like test data, shape (n_samples_test, n_features)
		pipeline: fitted model pipeline (PCA, CCA, CPCA, or UMAP)
		Y: optional labels (needed for supervised CCA or CPCA background)
		
	Returns:
		X_reduced: transformed test data
	"""
	if isinstance(pipeline, Pipeline):  # only pure PCA used a Pipeline object
		X_reduced = pipeline.transform(X)
		
	elif 'cca' in pipeline:
		X_scaled = pipeline['scaler'].transform(X)
		X_pca = pipeline['pca'].transform(X_scaled)
		X_reduced = pipeline['cca'].transform(X_pca)

	elif 'cpca' in pipeline:
		X_scaled = pipeline['scaler_X'].transform(X)
		X_pca = pipeline['pca'].transform(X_scaled)
		X_reduced = pipeline['cpca'].transform(X_pca,
						alpha_selection='manual', alpha_value=pipeline['alpha'])

	elif 'umap' in pipeline:
		X_scaled = pipeline['scaler'].transform(X)
		X_pca = pipeline['pca'].transform(X_scaled)
		X_reduced = pipeline['umap'].transform(X_pca)

	else:
		raise ValueError("Only PCA, CCA, CPCA and UMAP are supported right now.")
	
	return X_reduced

In [6]:
def evaluate_performance(classifier, X_train, y_train, X_test, y_test):
	"""
	Evaluate the performance of a fitted classifier on train and test data.
	"""
	y_train_pred = classifier.predict(X_train)	# binary
	y_train_prob = classifier.predict_proba(X_train)[:, 1]	# probabilities of task
	y_test_pred = classifier.predict(X_test)
	y_test_prob = classifier.predict_proba(X_test)[:, 1]

	train_accuracy = round(balanced_accuracy_score(y_train, y_train_pred), 3)
	train_auc = round(roc_auc_score(y_train, y_train_prob), 3)
	test_accuracy = round(balanced_accuracy_score(y_test, y_test_pred), 3)
	test_auc = round(roc_auc_score(y_test, y_test_prob), 3)	# evaluates how well the
	# model predicts the probability of the positive class (1 = task). 
 	# for this AUC, 0.5 = random guess
	return train_accuracy, train_auc, test_accuracy, test_auc

In [7]:
def classification_pipeline(
	X, y,
	model_class, n_components,
	classifier_type='logistic',
	classifier_kwargs=None,
	n_splits=5,
	alpha_list=None,
	random_state=42,
	verbose=True
	):
	"""
	Cross-validate a dimensionality reduction + classifier pipeline.

	Supports PCA, CCA, CPCA, and UMAP with Logistic Regression or SVM.
	For CPCA, the function searches over multiple alpha values and selects the best one
	per fold based on test accuracy.

	Parameters:
		X, y: input data and binary labels
		model_class: PCA, CCA, CPCA, or UMAP
		n_components: number of components to reduce to
		classifier_type: 'logistic' or 'svm'
		classifier_kwargs: optional keyword arguments (in a dictionary) for classifier
		n_splits: number of CV folds
		alpha_list: list of alpha values if using CPCA
		random_state: for reproducibility
		verbose: if True, prints progress

	Prints:
		A dictionary storing average train and test accuracy and AUC across folds.
	"""
	skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
	train_accs, train_aucs, test_accs, test_aucs = [], [], [], []	
 	# stores accuracy and AUC scores for each fold

	if classifier_kwargs is None:	# default empty dictionary for classifier parameters
		classifier_kwargs = {}

	for fold_idx, (train_idx, test_idx) in enumerate(skf.split(X, y)):
		X_train, X_test = X[train_idx], X[test_idx]
		y_train, y_test = y[train_idx], y[test_idx]

		if model_class.__name__ == 'CPCA':
			best_acc = 0
			best_X_train_reduced = None
			best_X_test_reduced = None
			best_alpha = None
   
			if alpha_list is None:
				alpha_list = [0.1, 1, 10]
	
			for alpha in alpha_list:
				X_train_rest = X_train[y_train == 0]
				X_train_reduced, pipeline = dim_reduction_train(
					X_train, model_class, n_components=n_components,
					Y=X_train_rest, alpha=alpha
				)
				X_test_reduced = dim_reduction_test(X_test, pipeline)

				if classifier_type == 'logistic':
					clf = LogisticRegression(random_state=random_state, 
							**classifier_kwargs)
				elif classifier_type == 'svm':
					clf = SVC(probability=True, random_state=random_state, 
			   				**classifier_kwargs)
				else:
					raise ValueError("Unsupported classifier type. Only 'logistic' and 'svm'.")

				clf.fit(X_train_reduced, y_train)
				y_pred = clf.predict(X_test_reduced)
				acc = balanced_accuracy_score(y_test, y_pred)

				if acc > best_acc:
					best_acc = acc
					best_X_train_reduced = X_train_reduced
					best_X_test_reduced = X_test_reduced
					best_alpha = alpha

			X_train_reduced = best_X_train_reduced
			X_test_reduced = best_X_test_reduced
   
			if verbose:
				print(f"Fold {fold_idx+1}: Best alpha = {best_alpha}")

		else:
			# PCA, CCA, or UMAP
			if model_class.__name__ == 'CCA':
				# will do one-hot encoding to 2 features
				if n_components > len(np.unique(y_train)):
					raise ValueError("Number of components for CCA cannot",
					  "exceed minimum number of features between X and Y.")
				Y_train = y_train
			else:
				Y_train = None

			X_train_reduced, pipeline = dim_reduction_train(
				X_train, model_class, n_components=n_components, Y=Y_train)
			X_test_reduced = dim_reduction_test(X_test, pipeline)

		if classifier_type == 'logistic':
			clf = LogisticRegression(random_state=random_state, **classifier_kwargs)
		elif classifier_type == 'svm':
			clf = SVC(probability=True, random_state=random_state, **classifier_kwargs)
		else:
			raise ValueError("Unsupported classifier type")

		clf.fit(X_train_reduced, y_train)

		train_accuracy, train_auc, test_accuracy, test_auc = evaluate_performance(
			clf, X_train_reduced, y_train, X_test_reduced, y_test
		)
  
		train_accs.append(train_accuracy)
		train_aucs.append(train_auc)
		test_accs.append(test_accuracy)
		test_aucs.append(test_auc)

		if verbose:
			print(f"Fold {fold_idx+1}: Train Accuracy={train_accs[-1]}, Train",
		 			f"AUC={train_aucs[-1]}, Test Accuracy={test_accs[-1]},",
					f"Test AUC={test_aucs[-1]}")

	scores = {	# balanced accuracies across task vs rest classes
		'train_accuracy': round(np.mean(train_accs), 3),
		'train_auc': round(np.mean(train_aucs), 3),
		'test_accuracy': round(np.mean(test_accs), 3),
		'test_auc': round(np.mean(test_aucs), 3)
	}
	print(scores)


In [15]:
# Parameters for the classifiers (modify as needed)

svm_params = {
	'kernel': 'rbf',
	'C': 1.0,
	'gamma': 'scale'
}

logistic_params = {
	'penalty': 'l1', 
	'solver': 'saga', 
	'max_iter': 5000
}

In [9]:
# PCA + Logistic Regression with penalty
# 328 components explains 90% variance in the dataset
classification_pipeline(
	X, y, model_class=PCA, n_components=328,
	classifier_type='logistic', classifier_kwargs=logistic_params,
	n_splits=5, random_state=42, verbose=True
)

Fold 1: Train Accuracy=0.675, Train AUC=0.728, Test Accuracy=0.651, Test AUC=0.69
Fold 2: Train Accuracy=0.675, Train AUC=0.728, Test Accuracy=0.647, Test AUC=0.681
Fold 3: Train Accuracy=0.675, Train AUC=0.727, Test Accuracy=0.664, Test AUC=0.69
Fold 4: Train Accuracy=0.677, Train AUC=0.73, Test Accuracy=0.639, Test AUC=0.681
Fold 5: Train Accuracy=0.679, Train AUC=0.732, Test Accuracy=0.65, Test AUC=0.684
{'train_accuracy': 0.676, 'train_auc': 0.729, 'test_accuracy': 0.65, 'test_auc': 0.685}


In [10]:
# PCA + SVM
classification_pipeline(
	X, y, model_class=PCA, n_components=328,
	classifier_type='svm', classifier_kwargs=svm_params,
	n_splits=5, random_state=42, verbose=True
)

Fold 1: Train Accuracy=0.733, Train AUC=0.89, Test Accuracy=0.712, Test AUC=0.778
Fold 2: Train Accuracy=0.735, Train AUC=0.892, Test Accuracy=0.7, Test AUC=0.771
Fold 3: Train Accuracy=0.73, Train AUC=0.892, Test Accuracy=0.707, Test AUC=0.796
Fold 4: Train Accuracy=0.736, Train AUC=0.893, Test Accuracy=0.705, Test AUC=0.781
Fold 5: Train Accuracy=0.735, Train AUC=0.891, Test Accuracy=0.717, Test AUC=0.773
{'train_accuracy': 0.734, 'train_auc': 0.892, 'test_accuracy': 0.708, 'test_auc': 0.78}


In [17]:
# PCA + SVM (ONLY 100 COMPONENTS)
classification_pipeline(
	X, y, model_class=PCA, n_components=100,
	classifier_type='svm', classifier_kwargs=svm_params,
	n_splits=5, random_state=42, verbose=True
)

Fold 1: Train Accuracy=0.717, Train AUC=0.836, Test Accuracy=0.703, Test AUC=0.747
Fold 2: Train Accuracy=0.72, Train AUC=0.84, Test Accuracy=0.686, Test AUC=0.735
Fold 3: Train Accuracy=0.717, Train AUC=0.833, Test Accuracy=0.7, Test AUC=0.758
Fold 4: Train Accuracy=0.717, Train AUC=0.836, Test Accuracy=0.698, Test AUC=0.749
Fold 5: Train Accuracy=0.717, Train AUC=0.833, Test Accuracy=0.704, Test AUC=0.75
{'train_accuracy': 0.718, 'train_auc': 0.836, 'test_accuracy': 0.698, 'test_auc': 0.748}


In [16]:
# PCA + SVM (ONLY 2 COMPONENTS)
classification_pipeline(
	X, y, model_class=PCA, n_components=2,
	classifier_type='svm', classifier_kwargs=svm_params,
	n_splits=5, random_state=42, verbose=True
)

Fold 1: Train Accuracy=0.597, Train AUC=0.61, Test Accuracy=0.593, Test AUC=0.592
Fold 2: Train Accuracy=0.602, Train AUC=0.614, Test Accuracy=0.578, Test AUC=0.592
Fold 3: Train Accuracy=0.595, Train AUC=0.608, Test Accuracy=0.599, Test AUC=0.626
Fold 4: Train Accuracy=0.595, Train AUC=0.61, Test Accuracy=0.6, Test AUC=0.611
Fold 5: Train Accuracy=0.593, Train AUC=0.61, Test Accuracy=0.61, Test AUC=0.622
{'train_accuracy': 0.596, 'train_auc': 0.61, 'test_accuracy': 0.596, 'test_auc': 0.609}


In [12]:
# Supervised CCA + Logistic Regression with penalty
# Note: supervised CCA can only keep a maximum of 2 components.
classification_pipeline(
	X, y, model_class=CCA, n_components=2,
	classifier_type='logistic', classifier_kwargs=logistic_params,
	n_splits=5, random_state=42, verbose=True
)

Fold 1: Train Accuracy=0.739, Train AUC=0.825, Test Accuracy=0.667, Test AUC=0.734
Fold 2: Train Accuracy=0.736, Train AUC=0.826, Test Accuracy=0.674, Test AUC=0.737
Fold 3: Train Accuracy=0.738, Train AUC=0.824, Test Accuracy=0.691, Test AUC=0.748
Fold 4: Train Accuracy=0.742, Train AUC=0.826, Test Accuracy=0.664, Test AUC=0.727
Fold 5: Train Accuracy=0.738, Train AUC=0.823, Test Accuracy=0.683, Test AUC=0.738
{'train_accuracy': 0.739, 'train_auc': 0.825, 'test_accuracy': 0.676, 'test_auc': 0.737}


In [13]:
# Supervised CCA + SVM
# Note: supervised CCA can only keep a maximum of 2 components.
classification_pipeline(
	X, y, model_class=CCA, n_components=2,
	classifier_type='svm', classifier_kwargs=svm_params,
	n_splits=5, random_state=42, verbose=True
)

Fold 1: Train Accuracy=0.733, Train AUC=0.801, Test Accuracy=0.683, Test AUC=0.719
Fold 2: Train Accuracy=0.733, Train AUC=0.798, Test Accuracy=0.677, Test AUC=0.718
Fold 3: Train Accuracy=0.735, Train AUC=0.796, Test Accuracy=0.698, Test AUC=0.739
Fold 4: Train Accuracy=0.737, Train AUC=0.803, Test Accuracy=0.675, Test AUC=0.708
Fold 5: Train Accuracy=0.735, Train AUC=0.803, Test Accuracy=0.685, Test AUC=0.728
{'train_accuracy': 0.735, 'train_auc': 0.8, 'test_accuracy': 0.684, 'test_auc': 0.722}


In [ ]:
# cPCA + Logistic Regression with penalty
classification_pipeline(
	X, y, model_class=CPCA, n_components=328,
	classifier_type='logistic', classifier_kwargs=logistic_params,
	n_splits=5, random_state=42, verbose=True
)

In [14]:
# cPCA + SVM
classification_pipeline(
	X, y, model_class=CPCA, n_components=328,
	classifier_type='svm', classifier_kwargs=svm_params,
	n_splits=5, random_state=42, verbose=True
)

Fold 1: Best alpha = 0.1
Fold 1: Train Accuracy=0.896, Train AUC=0.965, Test Accuracy=0.5, Test AUC=0.536
Fold 2: Best alpha = 0.1
Fold 2: Train Accuracy=0.894, Train AUC=0.967, Test Accuracy=0.5, Test AUC=0.578
Fold 3: Best alpha = 0.1
Fold 3: Train Accuracy=0.894, Train AUC=0.966, Test Accuracy=0.5, Test AUC=0.544
Fold 4: Best alpha = 0.1
Fold 4: Train Accuracy=0.895, Train AUC=0.965, Test Accuracy=0.5, Test AUC=0.507
Fold 5: Best alpha = 0.1
Fold 5: Train Accuracy=0.89, Train AUC=0.964, Test Accuracy=0.5, Test AUC=0.533
{'train_accuracy': 0.894, 'train_auc': 0.965, 'test_accuracy': 0.5, 'test_auc': 0.54}


In [ ]:
# cPCA + SVM
classification_pipeline(
	X, y, model_class=CPCA, n_components=328,
	classifier_type='svm', classifier_kwargs=svm_params,
	n_splits=5, random_state=42, verbose=True
)

Fold 1: Best alpha = 0.1
Fold 1: Train Accuracy=0.896, Train AUC=0.966, Test Accuracy=0.549, Test AUC=0.537


In [ ]:
# UMAP (non-linear, 50 components) + SVM
classification_pipeline(
	X, y, model_class=umap.UMAP, n_components=50,
	classifier_type='svm', classifier_kwargs=svm_params,
	n_splits=5, random_state=42, verbose=True
)

In [39]:
# CPCA + Logistic Regression with penalty

# First, split!
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Dimensionality reduction
X_train_rest = X_train[y_train == 0]  # background dataset for cPCA
X_train_reduced, pipeline = dim_reduction_train(X_train, CPCA, n_components=2, 
												Y=X_train_rest, alpha=0.5)
X_test_reduced = dim_reduction_test(X_test, pipeline)   # project test data on 
# learned subspace, so don't provide background Y=X_test_rest or else data leakage

# Classification model training
classifier = LogisticRegression(penalty='l1', solver='saga', max_iter=5000)
classifier.fit(X_train_reduced, y_train)

# Evaluation metrics
evaluate_performance(classifier, X_train_reduced, y_train, X_test_reduced, y_test)

Train accuracy: 0.5381
Train AUC: 0.5102
Test accuracy: 0.5264
Test AUC: 0.5289


In [45]:
# CPCA + Logistic Regression with penalty WITH 100 COMPONENTS

# First, split!
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Dimensionality reduction
X_train_rest = X_train[y_train == 0]  # background dataset for cPCA
X_train_reduced, pipeline = dim_reduction_train(X_train, CPCA, n_components=100, 
												Y=X_train_rest, alpha=0.5)
X_test_reduced = dim_reduction_test(X_test, pipeline)   # project test data on 
# learned subspace, so don't provide background Y=X_test_rest or else data leakage

# Classification model training
classifier = LogisticRegression(penalty='l1', solver='saga', max_iter=5000)
classifier.fit(X_train_reduced, y_train)

# Evaluation metrics
evaluate_performance(classifier, X_train_reduced, y_train, X_test_reduced, y_test)

Train accuracy: 0.5625
Train AUC: 0.6293
Test accuracy: 0.5657
Test AUC: 0.5891


In [38]:
# UMAP + Logistic Regression with penalty

# First, split!
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Dimensionality reduction
X_train_reduced, pipeline = dim_reduction_train(X_train, umap.UMAP, n_components=2)
X_test_reduced = dim_reduction_test(X_test, pipeline)

# Classification model training
classifier = LogisticRegression(penalty='l1', solver='saga', max_iter=5000)
classifier.fit(X_train_reduced, y_train)

# Evaluation metrics
evaluate_performance(classifier, X_train_reduced, y_train, X_test_reduced, y_test)

/Users/kinichen/miniconda3/envs/neuro_jb_lab_env/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Train accuracy: 0.6053
Train AUC: 0.5729
Test accuracy: 0.6218
Test AUC: 0.5937
